# Comparison of spectrum between eNATL60 outputs and Alitka Satellite 

Requisites :
 - git clone https://github.com/auraoupa/gonzag_cloud
 - git clone https://github.com/brodeau/climporn
 
Assumes that time is encoded in datetime64 format in both datasets.

Looks for model months in the longest year of Altika serie

In [1]:
import sys,os
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [11]:
GONZAG_DIR = '/home/jovyan/gonzag_cloud/gonzag' ; # get it there: https://github.com/brodeau/climporn
sys.path.append(GONZAG_DIR)
import gonzag as gz


## Params

For the model

In [3]:
model = 'GIGATL'
region = '1'
season = 'fma'
name_mod = model+'-Region'+region+'-'+season
name_ssh_mod = 'zeta'
name_lsm_mod = 'salt' 
name_lat_mod = 'lat'
name_lon_mod = 'lon'
name_time_mod = 'ocean_time'
l_griddist = False
period_mod = ['2014-02-01','2014-04-30']

For altimetry data

In [4]:
name_sat= 'Altika'
tag_sat_catalog = 'al'
name_ssh_sat='sla_unfiltered'
name_time_sat='time'
period_sat = ['2014-02-01','2014-04-30'] #should cover the same period of a different year if needed

## Data

In [5]:
import xarray as xr
import intake
import gcsfs

gcs = gcsfs.GCSFileSystem(requester_pays=True)

cat = intake.open_catalog('../SWOT-AdAC/catalog.yaml')
dsmod = cat[model](region=region,depth='surf', season=season).to_dask()

In [6]:
dsmod

<xarray.Dataset>
Dimensions:     (eta_rho: 1489, eta_v: 1488, time: 2280, xi_rho: 1491, xi_u: 1490)
Dimensions without coordinates: eta_rho, eta_v, time, xi_rho, xi_u
Data variables: (12/13)
    angle       (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    f           (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    h           (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    lat         (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    lon         (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    ocean_time  (time) float32 dask.array<chunksize=(24,), meta=np.ndarray>
    ...          ...
    pn          (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    salt        (time, eta_rho, xi_rho) float32 dask.array<chunksize=(24, 1489, 1491), meta=np.ndarray>
    temp        (time, eta_rho, xi_rho) float32 dask.array<chunksize=(24, 1489, 1491), meta=np.ndarray>
    u           (time, eta_rho, xi_u) float32 dask.array<chunksize=(24, 1489, 1490), meta=np.ndarray>
    v           (time, eta_v, xi_rho) float32 dask.array<chunksize=(24, 1488, 1491), meta=np.ndarray>
    zeta        (time, eta_rho, xi_rho) float32 dask.array<chunksize=(24, 1489, 1491), meta=np.ndarray>

In [7]:
from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/altimetry.yaml")
dssat  = cat[tag_sat_catalog].to_dask()

In [8]:
dssat

<xarray.Dataset>
Dimensions:         (time: 34340107)
Coordinates:
    latitude        (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    longitude       (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
  * time            (time) datetime64[ns] 2013-03-14T05:44:50.445185 ... 2015...
Data variables:
    cycle           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
    dac             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    lwe             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    mdt             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    ocean_tide      (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_filtered    (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_unfiltered  (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    track           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
Attributes: (12/24)
    Conventions:               CF-1.6
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    cdm_data_type:             Swath
    comment:                   Sea surface height measured by altimeters refe...
    contact:                   servicedesk.cmems@mercator-ocean.eu
    creator_email:             servicedesk.cmems@mercator-ocean.eu
    ...                        ...
    software_version:          6.2_DUACS_DT2018_baseline
    source:                    Altika measurements
    ssalto_duacs_comment:      The reference mission used for the altimeter i...
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    summary:                   SSALTO/DUACS Delayed-Time Level-3 sea surface ...
    title:                     DT Altika Global Ocean Along track SSALTO/DUAC...

In [ ]:
ModelGrid = gz.ModGrid( dsmod, period_mod, name_lon_mod, name_lat_mod , name_time_mod, dsmod, name_lsm_mod, distorded_grid=False )


In [9]:
dataset=dsmod
period=period_mod
varlon=name_lon_mod
varlat=name_lat_mod
vartime=name_time_mod
gridset=dsmod
varlsm=name_lsm_mod
distorded_grid=False

In [12]:
       from gonzag.io import GetTimeVector, GetModelCoor, GetModelLSM, Save2Dfield


In [16]:
rvt = GetTimeVector( dataset, name_time_mod, lquiet=True )

In [18]:
rvt.values

array([9.4910573e+08, 9.4910938e+08, 9.4911296e+08, ..., 9.5730298e+08,
       9.5730656e+08, 9.5731014e+08], dtype=float32)

In [19]:
import numpy as nmp
(rtu1,rtu2)=nmp.array(period, dtype='datetime64')


In [20]:
rtu1

numpy.datetime64('2014-02-01')

In [21]:
rtu2

numpy.datetime64('2014-04-30')

In [ ]:
jt1, jt2 = scan_idx( rvt, rtu1, rtu2 )

In [32]:
from datetime import datetime as dtm
if type(rvt.values[0]) == nmp.float32:
    rvtdt=[]
    for k in nmp.arange(len(rvt)):
        rvtdt.append(nmp.array(dtm.utcfromtimestamp(rvt[k]), dtype='datetime64')) 

In [36]:
dtm.utcfromtimestamp(rvt[0])

datetime.datetime(2000, 1, 29, 0, 28, 48)

In [33]:
rvtdt[0]

array('2000-01-29T00:28:48.000000', dtype='datetime64[us]')

In [34]:
    idx1=nmp.where(rvtdt>rtu1)
    idx2=nmp.where(rvtdt<rtu2)


In [35]:
idx1

(array([], dtype=int64),)

## Time overlap

In [26]:
import numpy as nmp
(date1,date2) = nmp.array(period, dtype='datetime64')
print(' *** The chosen period is from ',date1,' to ',date2)


 *** The chosen period is from  2014-02-01  to  2014-04-30


In [ ]:
        from .io import GetTimeVector, GetModelCoor, GetModelLSM, Save2Dfield


        self.file = dataset

        rvt = GetTimeVector( dataset, name_time_sat, lquiet=True )
        jt1, jt2 = scan_idx( rvt, rtu1, rtu2 )

In [ ]:
ModelGrid = gz.ModGrid( dsmod, date1 , date2, name_lon_mod, name_lat_mod , name_time_mod, dsmod, name_lsm_mod, distorded_grid=False )


In [9]:
(Nts,Ntm) = gz.GetTimeSize( dssat , dsmod, name_time_sat, name_time_mod, date1, date2 )
    print(' *** Time overlap between model and satellite in UNIX epoch time: it1, it2',date1,'--',date2)


 *** Time overlap between model and satellite in UNIX epoch time: it1, it2 2014-01-29 -- 2014-05-03


In [10]:
dataset_sat=dssat
dataset_mod=dsmod
timevar_sat=name_time_sat
timevar_mod=name_time_mod

In [11]:
    import numpy as nmp
    import pandas as pd
    from gonzag.io import GetTimeInfo
    #
    nts, range_sat = GetTimeInfo( dataset_sat, timevar_sat )
    ntm, range_mod = GetTimeInfo( dataset_mod, timevar_mod )


In [12]:
nts

34340107

In [13]:
ntm

2280

In [14]:
range_sat

(numpy.datetime64('2013-03-14T05:44:50.445185000'),
 numpy.datetime64('2015-03-31T22:41:36.909481000'))

In [15]:
range_mod

(datetime.datetime(2000, 1, 29, 0, 28, 48),
 datetime.datetime(2000, 5, 2, 23, 29, 4))

In [16]:
dataset=dataset_mod
varname=timevar_mod

In [18]:
    from datetime import datetime as dtm
    
    nt = dataset[varname].size
    clndr = dataset[varname]
 

In [19]:
print(nt)

2280


In [21]:
print(clndr.values)

[9.4910573e+08 9.4910938e+08 9.4911296e+08 ... 9.5730298e+08 9.5730656e+08
 9.5731014e+08]


In [23]:
type(clndr.values[0])

numpy.float32

In [24]:
    if type(clndr.values[0]) == nmp.float32:
        dt1 = dtm.utcfromtimestamp(clndr[0]) ; dt2 = dtm.utcfromtimestamp(clndr[nt-1])
    else:
        dt1 = clndr[0].values ; dt2 = clndr[nt-1].values


In [25]:
dt1

datetime.datetime(2000, 1, 29, 0, 28, 48)

In [26]:
dt2

datetime.datetime(2000, 5, 2, 23, 29, 4)

In [31]:
dtm.utcfromtimestamp(clndr[0].values.astype(int))

datetime.datetime(2000, 1, 29, 0, 28, 48)

In [30]:
clndr[0].values.astype(int)

array(949105728)

In [32]:
t=dsmod.ocean_time

In [33]:
dtm.utcfromtimestamp(t[0])

datetime.datetime(2000, 1, 29, 0, 28, 48)